In [25]:
import csv
import mmh3
from bitarray import bitarray
import math
import time

In [26]:
# size = 100000
# bit_array = bitarray(size)

In [27]:
n = 20000
p = 0.05
m = -(n * math.log(p))/(math.log(2)**2)
k = (m/n) * math.log(2) 
size = int(m)

In [28]:
size

124704

In [29]:
k

4.321928094887363

In [30]:
bit_array = bitarray(size)

In [60]:
start_time = time.time()
bit_array.setall(0)

def mapf(url):
    b1 = mmh3.hash(url, 1) % size
    bit_array[b1]=1

    b2 = mmh3.hash(url, 2) % size
    bit_array[b2]=1

    b3 = mmh3.hash(url, 3) % size
    bit_array[b3]=1

    b4 = mmh3.hash(url, 4) % size
    bit_array[b4]=1

#     b5 = mmh3.hash(url, 5) % size
#     bit_array[b5]=1

#     b6 = mmh3.hash(url, 6) % size
#     bit_array[b6]=1

#     b7 = mmh3.hash(url, 7) % size
#     bit_array[b7]=1


r = csv.reader(open("new_data.csv"))
for row in r:
    url=row[1]
    mapf(url)
print("Time for build --- %s seconds ---" % (time.time() - start_time))

Time for build --- 0.07630491256713867 seconds ---


In [61]:
start_time = time.time()
disk_accesses=0;

ba = bit_array

def check(url):
    global disk_accesses
    b1 = mmh3.hash(url, 1) % size
    b2 = mmh3.hash(url, 2) % size
    b3 = mmh3.hash(url, 3) % size
    b4 = mmh3.hash(url, 4) % size
#     b5 = mmh3.hash(url, 5) % size
#     b6 = mmh3.hash(url, 6) % size
#     b7 = mmh3.hash(url, 7) % size
    if(ba[b1]==True 
    and ba[b2]==True 
    and ba[b3]==True 
    and ba[b4]==True 
#     and ba[b5]==True 
#     and ba[b6]==True 
#     and ba[b7]==True
      ):
        disk_accesses=disk_accesses+1



r = csv.reader(open("top2m.csv"))

for row in r:
    url=row[1]
    check(url)
print(disk_accesses);

print("Time for check --- %s seconds ---" % (time.time() - start_time))

245
Time for check --- 0.004625082015991211 seconds ---


In [58]:
start_time = time.time()
link = csv.reader(open("new_data.csv"))
data=[]
for item in link:
    data.append(item[1])
test = csv.reader(open("top2m.csv"))
count=0
for number,item in test:
    if item in data:
        count+=1
print(count)
print("Time for check --- %s seconds ---" % (time.time() - start_time))

200
Time for check --- 0.2752988338470459 seconds ---


200